In [2]:
!pip install pydantic requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 389.7 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 295.1 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 452.7 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 912.4 kB/s eta 0:00:000:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.8.0
    Uninstalling typing_extensions-4.8.0:
      Successfully uninstalled typing_extensions-4.8.0


In [4]:
!pip freeze

absl-py==2.0.0
alembic @ file:///home/conda/feedstock_root/build_artifacts/alembic_1694690212886/work
altair @ file:///home/conda/feedstock_root/build_artifacts/altair_1696364485230/work
annotated-types==0.7.0
anyio @ file:///home/conda/feedstock_root/build_artifacts/anyio_1693488585952/work
argon2-cffi @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi_1692818318753/work
argon2-cffi-bindings @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi-bindings_1695386553988/work
arrow @ file:///home/conda/feedstock_root/build_artifacts/arrow_1696128962909/work
asttokens @ file:///home/conda/feedstock_root/build_artifacts/asttokens_1694046349000/work
astunparse==1.6.3
async-generator==1.10
async-lru @ file:///home/conda/feedstock_root/build_artifacts/async-lru_1690563019058/work
attrs @ file:///home/conda/feedstock_root/build_artifacts/attrs_1683424013410/work
Babel @ file:///home/conda/feedstock_root/build_artifacts/babel_1696976437817/work
backcall @ file:///home/cond

In [3]:
!pip install pydantic-ai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.5/210.5 kB 204.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 kB 426.4 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 194.1 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 547.8 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 194.8 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 107.4 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.1/216.1 kB 305.3 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 kB 276.2 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 206.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 297.4 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 496.2 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━

In [7]:
import requests
import json
from pydantic import BaseModel, Field

# ──────────────────────────────────────────────────────────────────────────────
# 1) Define your schemas
# ──────────────────────────────────────────────────────────────────────────────

class QueryInput(BaseModel):
    """What to fetch—could be SQL, ElasticDSL, a search term, etc."""
    query: str = Field(..., description="SQL query or search keyword")

class QueryOutput(BaseModel):
    """Raw rows or records as a list of dicts."""
    rows: list[dict] = Field(..., description="List of data records")

class ReportInput(BaseModel):
    """Takes the fetched rows and emits a formatted report."""
    rows: list[dict]

class ReportOutput(BaseModel):
    """A plain-text summary for end-users."""
    report: str = Field(..., description="Summary report")

class AnalysisInput(BaseModel):
    """Takes the same raw rows and emits structured insights."""
    rows: list[dict]

class AnalysisOutput(BaseModel):
    """List of key insights, each as a string."""
    insights: list[str] = Field(..., description="Actionable insights")

# ──────────────────────────────────────────────────────────────────────────────
# 2) Custom LLM function
# ──────────────────────────────────────────────────────────────────────────────

def custom_llm(prompt: str) -> str:
    """Call local Ollama API with streaming response handling"""
    url = "http://host.docker.internal:11434/api/generate"
    data = {
        "model": "qwen2:1.5b",
        "prompt": 'به فارسی جواب بده ' + prompt
    }
    
    try:
        response = requests.post(url, json=data, stream=True, timeout=30)
        response.raise_for_status()  # Raise HTTP errors
        
        final_output = ""
        for line in response.iter_lines():
            if line:  # Filter out keep-alive new lines
                try:
                    json_data = json.loads(line.decode("utf-8"))
                    final_output += json_data.get("response", "")
                except json.JSONDecodeError:
                    continue  # Skip invalid JSON lines
        return final_output
    except requests.RequestException as e:
        return f"LLM Error: {str(e)}"

# ──────────────────────────────────────────────────────────────────────────────
# 3) Implement each agent
# ──────────────────────────────────────────────────────────────────────────────

class QueryAgent:
    """Fetches data from any external source (DB, REST, etc.)."""
    def run(self, inputs: QueryInput) -> QueryOutput:
        # Fake database - replace with real data source
        fake_db = [
            {"region": "EMEA", "sales": 1_000, "product": "A"},
            {"region": "APAC", "sales": 1_500, "product": "B"},
            {"region": "NA",   "sales": 2_000, "product": "A"},
            {"region": "EMEA", "sales": 800,  "product": "C"},
        ]
        # Filter by query
        rows = [r for r in fake_db if inputs.query.lower() in r["region"].lower()]
        return QueryOutput(rows=rows)

class ReportAgent:
    """Turns raw rows into a prose report via LLM."""
    def run(self, inputs: ReportInput) -> ReportOutput:
        prompt = f"""
        You are a sales analyst assistant. Write a concise one-paragraph summary
        of these sales records in Persian: {inputs.rows}
        """
        summary = custom_llm(prompt).strip()
        return ReportOutput(report=summary)

class AnalysisAgent:
    """Generates a bulleted list of deeper insights."""
    def run(self, inputs: AnalysisInput) -> AnalysisOutput:
        prompt = f"""
        Given these sales records {inputs.rows},
        provide 3–5 bullet points of insights or recommended next steps in Persian.
        Format each insight on a new line starting with '• '.
        """
        raw_analysis = custom_llm(prompt).strip()
        
        # Parse bullet points
        insights = [
            line.strip() 
            for line in raw_analysis.split('\n') 
            if line.strip() and line.strip().startswith(('•', '-', '*', '‣'))
        ]
        
        # Fallback if bullet parsing fails
        if not insights:
            insights = [raw_analysis]
            
        return AnalysisOutput(insights=insights)

# ──────────────────────────────────────────────────────────────────────────────
# 4) Main execution flow
# ──────────────────────────────────────────────────────────────────────────────

def main():
    # Initialize agents
    qa = QueryAgent()
    ra = ReportAgent()
    aa = AnalysisAgent()

    # Get user query
    user_query = input("Enter region to analyze (e.g., EMEA, APAC, NA): ").strip()
    
    # Execute pipeline
    try:
        qout = qa.run(QueryInput(query=user_query))
        
        if not qout.rows:
            print("\n⚠️ No data found for this region")
            return
            
        rout = ra.run(ReportInput(rows=qout.rows))
        aout = aa.run(AnalysisInput(rows=qout.rows))

        # Display results
        print("\n" + "="*50)
        print(f"📊 SALES REPORT FOR: {user_query.upper()}")
        print("="*50)
        print("\n🔍 Summary:")
        print(rout.report)
        
        print("\n💡 Key Insights:")
        for i, insight in enumerate(aout.insights, 1):
            print(f"  {i}. {insight}")
            
    except Exception as e:
        print(f"\n❌ Error: {str(e)}")

if __name__ == "__main__":
    main()

Enter region to analyze (e.g., EMEA, APAC, NA):  EMEA



📊 SALES REPORT FOR: EMEA

🔍 Summary:
این دسته بازاریابی ها از ۱۰۰۰ فروش در منطقه EMEA و ۸۰۰ فروش دارای محصول C هستند.

💡 Key Insights:
  1. • هیچ بارهایی از محصولات A و C وجود نداشتند در EMEA کشور.
  2. • توزیع محصولات C بسیار ارزشمند است.
  3. • با توجه به تاثیر محصول C، شرکت میتواند محصول های دیگری را چسبنده و با ابزارهای مختلف بازاریابی داشته باشد.
  4. • با این حال، بررسی بیشتری در مورد این محصولات باید انجام شود تا بهتر قرار دهد.


In [8]:
import pandas as pd
import requests
import json
from pydantic import BaseModel, Field
from typing import List, Dict

# Load the telecom churn dataset
df = pd.read_csv('telecom_churn.csv')

# Display dataset information
print("Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"First 3 rows:\n{df.head(3)}")

# ──────────────────────────────────────────────────────────────────────────────
# Custom LLM function
# ──────────────────────────────────────────────────────────────────────────────

def custom_llm(prompt: str) -> str:
    """Call local Ollama API with streaming response handling"""
    url = "http://host.docker.internal:11434/api/generate"
    data = {
        "model": "qwen2:1.5b",
        "prompt": 'به فارسی جواب بده ' + prompt
    }
    
    try:
        response = requests.post(url, json=data, stream=True, timeout=120)
        response.raise_for_status()
        
        final_output = ""
        for line in response.iter_lines():
            if line:
                try:
                    json_data = json.loads(line.decode("utf-8"))
                    final_output += json_data.get("response", "")
                except json.JSONDecodeError:
                    continue
        return final_output
    except requests.RequestException as e:
        return f"LLM Error: {str(e)}"

# ──────────────────────────────────────────────────────────────────────────────
# Define schemas
# ──────────────────────────────────────────────────────────────────────────────

class QueryInput(BaseModel):
    query: str = Field(..., description="Query to filter telecom data")

class QueryOutput(BaseModel):
    rows: List[Dict] = Field(..., description="Filtered telecom records")

class ReportInput(BaseModel):
    rows: List[Dict]

class ReportOutput(BaseModel):
    report: str = Field(..., description="Summary report")

class AnalysisInput(BaseModel):
    rows: List[Dict]

class AnalysisOutput(BaseModel):
    insights: List[str] = Field(..., description="Actionable insights")

# ──────────────────────────────────────────────────────────────────────────────
# Implement agents
# ──────────────────────────────────────────────────────────────────────────────

class QueryAgent:
    """Fetches telecom churn data based on query"""
    def run(self, inputs: QueryInput) -> QueryOutput:
        try:
            # Convert query to lowercase for case-insensitive matching
            query = inputs.query.lower()
            
            # Filter based on different criteria
            if query == "all":
                filtered = df
            elif query == "churned":
                filtered = df[df['Churn'] == True]
            elif query == "active":
                filtered = df[df['Churn'] == False]
            elif len(query) == 2 and query.isalpha():  # State code
                filtered = df[df['State'].str.lower() == query]
            elif query.isdigit() and len(query) == 3:  # Area code
                filtered = df[df['Area code'] == int(query)]
            else:
                # Try to match in multiple columns
                filtered = df[
                    df['State'].str.lower().str.contains(query) |
                    df['International plan'].str.lower().str.contains(query) |
                    df['Voice mail plan'].str.lower().str.contains(query)
                ]
            
            # Convert to list of dictionaries and limit to 100 records
            records = filtered.head(100).to_dict('records')
            return QueryOutput(rows=records)
        
        except Exception as e:
            print(f"Query error: {str(e)}")
            return QueryOutput(rows=[])

class ReportAgent:
    """Generates telecom summary report"""
    def run(self, inputs: ReportInput) -> ReportOutput:
        prompt = f"""
        You are a telecom analyst. Write a concise summary in Persian about 
        these customer records. Focus on churn rates, key features, and patterns.
        Data: {inputs.rows[:3]}  # Show first 3 records as sample
        Total records: {len(inputs.rows)}
        """
        summary = custom_llm(prompt).strip()
        return ReportOutput(report=summary)

class AnalysisAgent:
    """Generates insights from telecom data"""
    def run(self, inputs: AnalysisInput) -> AnalysisOutput:
        prompt = f"""
        Analyze these telecom customer records and provide 3-5 key insights 
        in Persian with bullet points. Focus on churn patterns and recommendations.
        Start each insight with '• '. 
        Data sample: {inputs.rows[:2]}
        Total records: {len(inputs.rows)}
        """
        raw_analysis = custom_llm(prompt).strip()
        
        # Parse bullet points
        insights = [
            line.strip() 
            for line in raw_analysis.split('\n') 
            if line.strip().startswith(('•', '-', '*', '‣'))
        ]
        
        # Fallback if bullet parsing fails
        if not insights:
            insights = [raw_analysis]
            
        return AnalysisOutput(insights=insights)

# ──────────────────────────────────────────────────────────────────────────────
# Main execution flow
# ──────────────────────────────────────────────────────────────────────────────

def main():
    # Initialize agents
    qa = QueryAgent()
    ra = ReportAgent()
    aa = AnalysisAgent()
    
    # Display query options
    print("\n" + "═"*50)
    print("📞 Telecom Churn Analysis System")
    print("═"*50)
    print("Query options:")
    print("- 'all': All customers")
    print("- 'churned': Churned customers")
    print("- 'active': Active customers")
    print("- State code (e.g., 'ca', 'ny')")
    print("- Area code (e.g., '408', '510')")
    print("- Plan type (e.g., 'yes', 'no')")
    print("═"*50)
    
    # Get user query
    user_query = input("Enter your query: ").strip()
    
    # Execute pipeline
    try:
        print("\n⚡ Processing query...")
        qout = qa.run(QueryInput(query=user_query))
        
        if not qout.rows:
            print("\n⚠️ No data found for this query")
            return
            
        print(f"✅ Found {len(qout.rows)} records")
        
        print("\n📝 Generating report...")
        rout = ra.run(ReportInput(rows=qout.rows))
        
        print("\n🔍 Analyzing insights...")
        aout = aa.run(AnalysisInput(rows=qout.rows))

        # Display results
        print("\n" + "="*50)
        print(f"📊 TELECOM ANALYSIS REPORT: {user_query.upper()}")
        print("="*50)
        print("\n📄 Summary Report:")
        print(rout.report)
        
        print("\n💡 Key Insights:")
        for i, insight in enumerate(aout.insights, 1):
            print(f"  {i}. {insight}")
            
    except Exception as e:
        print(f"\n❌ Error: {str(e)}")

if __name__ == "__main__":
    main()

FileNotFoundError: [Errno 2] No such file or directory: 'telecom_churn.csv'

In [12]:
import pandas as pd

# Load a CSV file
df = pd.read_csv('/work/telecom_churn.csv')

# Display the first few rows
print(df.head())


   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingMovies        Contract Pape

In [ ]:
import pandas as pd
import requests
import json
import re
from pydantic import BaseModel, Field
from typing import List, Dict, Optional

# Load telecom churn dataset
try:
    df = pd.read_csv('/work/telecom_churn.csv')
    print("✅ Dataset loaded successfully!")
    print(f"📊 Shape: {df.shape}")
    print(f"📝 Columns: {list(df.columns)}")
except FileNotFoundError:
    print("❌ Error: telecom_churn.csv file not found")
    exit()

# ──────────────────────────────────────────────────────────────────────────────
# Enhanced LLM function with caching and improved error handling
# ──────────────────────────────────────────────────────────────────────────────

LLM_CACHE = {}

def custom_llm(prompt: str, use_cache: bool = True) -> str:
    """Call local Ollama API with enhanced features"""
    # Check cache first
    if use_cache and prompt in LLM_CACHE:
        return LLM_CACHE[prompt]
    
    url = "http://host.docker.internal:11434/api/generate"
    data = {
        "model": "qwen2:1.5b",
        "prompt": 'به فارسی جواب بده ' + prompt,
        "options": {"temperature": 0.3, "num_ctx": 4096}
    }
    
    try:
        response = requests.post(url, json=data, stream=True, timeout=90)
        response.raise_for_status()
        
        final_output = ""
        for line in response.iter_lines():
            if line:
                try:
                    json_data = json.loads(line.decode("utf-8"))
                    if "response" in json_data:
                        final_output += json_data["response"]
                    if json_data.get("done", False):
                        break
                except json.JSONDecodeError:
                    continue
        
        # Cache the result
        LLM_CACHE[prompt] = final_output
        return final_output
    except requests.RequestException as e:
        return f"خطا در ارتباط با مدل زبانی: {str(e)}"

# ──────────────────────────────────────────────────────────────────────────────
# Analysis Agent with intelligent prompt generation
# ──────────────────────────────────────────────────────────────────────────────

class AnalysisInput(BaseModel):
    query: str = Field(..., description="Natural language query about the data")
    sample_size: int = Field(5, description="Number of sample rows to include")

class AnalysisOutput(BaseModel):
    insights: str = Field(..., description="Analysis results in Persian")
    visualization_suggestion: str = Field(..., description="Suggested visualization type")

class TelecomAnalysisAgent:
    """Intelligent agent for telecom churn data analysis"""
    
    def generate_analysis_prompt(self, query: str, sample: List[dict]) -> str:
        """Create optimized prompt for data analysis"""
        prompt = f"""
        شما یک تحلیل‌گر داده‌های مشترکین مخابراتی هستید. کاربر سوال زیر را پرسیده است:
        «{query}»
        
        داده‌های نمونه (۵ ردیف اول):
        {json.dumps(sample, ensure_ascii=False, indent=2)}
        
        ساختار کامل داده‌ها:
        {list(df.columns)}
        
        دستورالعمل:
        ۱. تحلیل را دقیقاً بر اساس سوال کاربر انجام دهید
        ۲. از اصطلاحات تخصصی مخابرات (مثل نرخ ریزش، ARPU، وفاداری مشتری) استفاده کنید
        ۳. نتایج را به فارسی ساده و قابل فهم ارائه دهید
        ۴. در انتها نوع نمودار مناسب برای نمایش نتایج را پیشنهاد دهید
        
        تحلیل و نتیجه‌گیری:
        """
        return prompt
    
    def generate_visualization_prompt(self, insight: str) -> str:
        """Suggest appropriate visualization based on insight"""
        prompt = f"""
        بر اساس این تحلیل: «{insight}»
        بهترین نوع نمودار برای نمایش این نتیجه کدام است؟ فقط نام نمودار را بگویید.
        
        گزینه‌ها:
        - نمودار میله‌ای
        - نمودار دایره‌ای
        - نمودار خطی
        - هیستوگرام
        - نمودار پراکندگی
        - باکس پلات
        - هیچکدام
        
        پاسخ:
        """
        return prompt
    
    def run(self, inputs: AnalysisInput) -> AnalysisOutput:
        # Get data sample
        sample = df.head(inputs.sample_size).to_dict('records')
        
        # Generate analysis prompt
        analysis_prompt = self.generate_analysis_prompt(inputs.query, sample)
        insight = custom_llm(analysis_prompt).strip()
        
        # Generate visualization suggestion
        viz_prompt = self.generate_visualization_prompt(insight)
        visualization = custom_llm(viz_prompt).strip()
        
        return AnalysisOutput(
            insights=insight,
            visualization_suggestion=visualization
        )

# ──────────────────────────────────────────────────────────────────────────────
# Query Understanding Agent
# ──────────────────────────────────────────────────────────────────────────────

class QueryUnderstandingAgent:
    """Improves user queries for better analysis"""
    
    def enhance_query(self, query: str) -> str:
        """Refine natural language queries"""
        prompt = f"""
        کاربر این سوال را درباره داده‌های مشترکین تلفن همراه پرسیده است:
        «{query}»
        
        این سوال را به صورت حرفه‌ای و دقیق‌تر بازنویسی کن تا برای تحلیل داده‌ها مناسب‌تر باشد. 
        از اصطلاحات تخصصی حوزه مخابرات استفاده کن.
        
        سوال بهبودیافته:
        """
        return custom_llm(prompt).strip()

# ──────────────────────────────────────────────────────────────────────────────
# Main Analysis System
# ──────────────────────────────────────────────────────────────────────────────

def analyze_telecom_data():
    # Initialize agents
    analysis_agent = TelecomAnalysisAgent()
    query_agent = QueryUnderstandingAgent()
    
    print("\n" + "="*50)
    print("📊 سیستم تحلیل داده‌های مشترکین مخابراتی")
    print("="*50)
    
    while True:
        # Get user query
        user_query = input("\n💬 سوال خود را وارد کنید (یا 'exit' برای خروج): ").strip()
        
        if user_query.lower() in ['exit', 'خروج']:
            break
            
        if not user_query:
            continue
            
        try:
            # Enhance the query
            enhanced_query = query_agent.enhance_query(user_query)
            print(f"\n🔍 سوال بهبودیافته: {enhanced_query}")
            
            # Perform analysis
            print("\n🔄 در حال تحلیل داده‌ها...")
            result = analysis_agent.run(AnalysisInput(query=enhanced_query))
            
            # Display results
            print("\n" + "="*50)
            print("💡 نتایج تحلیل:")
            print(result.insights)
            
            print("\n📊 پیشنهاد مصورسازی:")
            print(result.visualization_suggestion)
            print("="*50)
            
        except Exception as e:
            print(f"\n❌ خطا: {str(e)}")

if __name__ == "__main__":
    analyze_telecom_data()

✅ Dataset loaded successfully!
📊 Shape: (7043, 21)
📝 Columns: ['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']

📊 سیستم تحلیل داده‌های مشترکین مخابراتی



💬 سوال خود را وارد کنید (یا 'exit' برای خروج):  مشتریان ریزشی چه ویژگی‌هایی دارند؟



🔍 سوال بهبودیافته: "متاسفانه، به عنوان یک هوش مصنوعی من نمی‌توانم این سوال را درست بازنویسی کنم. لطفاً مشتریان ریزشی را بپرسید تا بتوانم به شما کمک کنم."

🔄 در حال تحلیل داده‌ها...

💡 نتایج تحلیل:
با توجه به سوال کاربر، شما می‌توانید از داده‌ها برای محاسبه نرخ ریزش مشتری استفاده کنید. در اینجا، نمایشگر چندگانه (Pie Chart) به عنوان نمودار مناسب برای نمایش نتایج را پیشنهاد می‌کنم.

برخی از نتایج:
- نرخ ریزش مشتری: 20%
- نرخ ریزش مشتری با استفاده از ARPU: 30%
- نمودار نرخ ریزش مشتری:

![نمودار نرخ ریزش مشتری](https://i.imgur.com/7Wn5bY1.png)

این نمودار با استفاده از ARPU، نموداری برای نمایش نرخ ریزش مشتری و تاثیر آن بر ARPU را ارائه می‌دهد.

📊 پیشنهاد مصورسازی:
نمودار دایره‌ای



💬 سوال خود را وارد کنید (یا 'exit' برای خروج):  مشتریان ریزشی چه ویژگی‌هایی دارند؟ با مثال



🔍 سوال بهبودیافته: "مشتریان ریزشی چه ویژگی‌هایی دارند؟ با مثال?"

🔄 در حال تحلیل داده‌ها...

💡 نتایج تحلیل:
با توجه به سوال کاربر، شما می‌توانید اطلاعاتی درباره مشتریان ریزشی را بررسی کنید. این مشتریان در یک دوره زمانی دسترسی به خدمات مخابراتی دارند و همچنین نسبت به مشتریان دیگر دارای تفاوت‌هایی هستند.

برای انجام این کار، شما باید از داده‌های موجود استفاده کنید. این داده‌ها شامل ۵ ردیف اولیه می‌باشند:

1. customerID
2. gender
3. SeniorCitizen
4. Partner
5. Dependents
6. tenure
7. PhoneService
8. MultipleLines
9. InternetService
10. OnlineSecurity
11. OnlineBackup
12. DeviceProtection
13. TechSupport
14. StreamingTV
15. StreamingMovies
16. Contract
17. PaperlessBilling
18. PaymentMethod
19. MonthlyCharges
20. TotalCharges
21. Churn

برای تحلیل این داده‌ها، شما می‌توانید از چندین روش مختلف استفاده کنید. برخی از این روش ها شامل:

1. تجزیه و تحلیل نمودار: این روش به منظور پردازش داده‌ها و تجزیه آن‌ها در مورد مشتریان ریزشی استفاده می‌شود.

2. تحلیل مدل‌های دستگاه‌های شناختی: این روش برای 